In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

#### 시험을 위해 일부 데이터만 사용해 본다.

In [3]:
x_train = newsgroups_train.data
x_test = newsgroups_test.data

y_train = newsgroups_train.target #20개 카테고리로 클래스가 분류됨
y_test = newsgroups_test.target

In [4]:
print ("List of all 20 categories:")
print (newsgroups_train.target_names)
print ("\n")
print ("Sample Email:")
print (x_train[0])
print ("Sample Target Category:")
print (y_train[0])
print (newsgroups_train.target_names[y_train[0]])

List of all 20 categories:
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Sample Email:
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is

#### Used for pre-processing data

In [5]:
import nltk
from nltk.corpus import stopwords #불용어
from nltk.stem import WordNetLemmatizer #형태소로 어근에 맞추어 단어 분류
import string
import pandas as pd
from nltk import pos_tag #lemma처리 해도 working과 같이 명사, 동사로 쓰이는 경우가 있으니 태그로 처리할거야
from nltk.stem import PorterStemmer

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
def preprocessing(text): #email 문서 1개
    text2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())
                                            #특수 문자를 공백 문자로 처리하겠다.
    #text2 = ''
    #for ch in text:
    #    if ch in string.punctuation:
    #        text2 += ' '
    #    else:
    #        text2 += ch
    #text2 = ' '.join(text2.split())  # '\n'을 제거하기 위해 split() 후 다시 합침
    
                                                 #Email 1개 문서
    tokens = [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
    tokens = [word.lower() for word in tokens]
    
    #tokens = []
    #for sent in nltk.sent_tokenize(text2):
    #    for word in nltk.word_tokenize(sent):
    #        tokens.append(word.lower())
            
        #sent: email 1개의 1문장
        #단어단위로 쪼개서 word안에 넣어
        #word를 소문자로 token이라는 list에 집어넣어
        
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    tokens = [word for word in tokens if len(word)>=3]
    
    #stemmer = PorterStemmer() #어간 추출
    #try:
    #    tokens = [stemmer.stem(word) for word in tokens]

    #except:
    #    tokens = tokens #혹시나 잘 안되면 냅둬라
        
    tagged_corpus = pos_tag(tokens)    #품사에 따라 태그 붙이기. 얘는 nltk에 내장되어 있다.
    
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']

    lemmatizer = WordNetLemmatizer() #아래 것들을 얘가 해주는거

    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n') #token = working
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v') #token = work
        else:
            return lemmatizer.lemmatize(token,'n')
    
    pre_proc_text =  " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus]) #함수 호출, 함수 안 함수
                                #여기서 불렀고         태그 붙인 것에 따라 하나씩 pre_proc_text에 넣어 
    return pre_proc_text

In [8]:
lemmatizer=WordNetLemmatizer()
lemmatizer.lemmatize('riding','v')

'rid'

In [9]:
x_train_preprocessed  = [] #전처리가 완료된 학습 시킬 데이터
for i in x_train:
    x_train_preprocessed.append(preprocessing(i))

In [10]:
x_test_preprocessed = []
for i in x_test:
    x_test_preprocessed.append(preprocessing(i))

In [11]:
x_train_preprocessed[0]

'lerxst wam umd edu thing subject car nntp post host rac3 wam umd edu organization university maryland college park line wonder anyone could enlighten car saw day door sport car look late 60 early 70 call bricklin door really small addition front bumper separate rest body know anyone tellme model name engine spec year production car make history whatever info funky look car please mail thanks bring neighborhood lerxst'

#### building TFIDF vectorizer 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 2),  stop_words='english', max_features= 10000)
    #적어도 최소 2번 이상 쓰인 단어만 쓴다.      (unigram, bigram) 이미 sklearn에 stopword 내장, 가장 빈도 높은 것으로 10000개를 끊어 사용하겠다.

        # = vectorizer.fit(x_train_preprocessed)-------tf x idf
        # = vectorizer.transform(x+train_processed).todense()
x_train_2 = vectorizer.fit_transform(x_train_preprocessed).todense()
#         [                         tf, idf               ] todense : 여기까지 CSR(행렬)로 리턴해줌.
x_test_2 = vectorizer.transform(x_test_preprocessed).todense()
                                #=tf

In [14]:
x_train_2[0]  #띄엄띄엄있다 -> sparse matrix(내부적으로 CSR format 가지고 있음.)

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [15]:
x_train_2[0].shape           

(1, 10000)

In [16]:
len(vectorizer.get_feature_names()) #voca의 크기

10000

#### Deep Learning modules

In [17]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


#### Definiting hyper parameters

In [18]:
np.random.seed(1337) 
nb_classes = 20 
batch_size = 64 #mini-batch
nb_epochs = 200

Y_train = np_utils.to_categorical(y_train, nb_classes) #one-hot으로 변환 (인공신경망 동작원리)
                                #y_train을 넣고 20개로 변환시켜라

#        y-train이 0이면 첫 번째만 1로 활성화시키고 나며지는 0으로 하겠다. 
#           0->[1,0,0,0,0,0,,,,,0,0]
#          1->[0,1,0,0,0,0,,,,0,0] 이런식으로 20개 총 19번까지 빼내겠다  

# 물론 2진법으로 변환해서 19=10011-> [1,0,0,1,1]이런식으로 넣을 수도 있음  (얘가 메모리는 더 작음)

In [19]:
print(Y_train[0])
print(Y_train.shape)

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(11314, 20)


#### Deep Layer Model building in Keras

In [20]:
model = Sequential()

model.add(Dense(1000, input_shape= (x_train_2.shape[1],)))
                                    #2000행(email 개수), shape[1]=10000개( feature는 단어 만개만 쓰기로 했으니까!)
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))#activationFunction은 softmax! 다중분류니까                           sigmoid도 가능 근데 이러면
model.compile(loss='categorical_crossentropy', optimizer='adam') #lossfuction은 크로스 엔트로피! 확률분포니까         얘는 MSE써
print (model.summary())
                                                    #10,000 x 1,000개 +1000개(bias)

W0712 13:29:41.183151  5396 deprecation_wrapper.py:119] From C:\Anaconda3.7\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0712 13:29:41.477168  5396 deprecation_wrapper.py:119] From C:\Anaconda3.7\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0712 13:29:41.536171  5396 deprecation_wrapper.py:119] From C:\Anaconda3.7\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0712 13:29:41.658178  5396 deprecation_wrapper.py:119] From C:\Anaconda3.7\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0712 13:29:41.693180  5396 deprecation.py:506] From C:\Anaconda3.7\lib\site-packages\

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              10001000  
_________________________________________________________________
activation_1 (Activation)    (None, 1000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                20020     
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
Total params: 10,021,020
Trainable params: 10,021,020
Non-trainable params: 0
_________________________________________________________________
None


#### Model Training

In [21]:
model.fit(x_train_2, Y_train, batch_size=batch_size, epochs=nb_epochs,verbose=1)

W0712 13:29:42.089203  5396 deprecation.py:323] From C:\Anaconda3.7\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
11314/11314 [==============================] - 31s 3ms/step - loss: 1.5118
Epoch 2/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.2931
Epoch 3/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.1159
Epoch 4/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0556
Epoch 5/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0303
Epoch 6/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0196
Epoch 7/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0137
Epoch 8/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0109
Epoch 9/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0089
Epoch 10/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0072
Epoch 11/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0063
Epoch 12/200
11314/11314 [===============

11314/11314 [==============================] - 28s 3ms/step - loss: 0.0030
Epoch 95/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0017
Epoch 96/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0015
Epoch 97/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0022
Epoch 98/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0014
Epoch 99/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0015
Epoch 100/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0020
Epoch 101/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0015
Epoch 102/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0035
Epoch 103/200
11314/11314 [==============================] - 28s 3ms/step - loss: 0.0010
Epoch 104/200
11314/11314 [==============================] - 29s 3ms/step - loss: 9.8757e-04
Epoch 105/200
11314/11314 [=========

11314/11314 [==============================] - 29s 3ms/step - loss: 0.0017
Epoch 186/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0012
Epoch 187/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0029
Epoch 188/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0020
Epoch 189/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0020
Epoch 190/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0016
Epoch 191/200
11314/11314 [==============================] - 29s 3ms/step - loss: 9.2052e-04
Epoch 192/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0019
Epoch 193/200
11314/11314 [==============================] - 29s 3ms/step - loss: 6.1801e-04
Epoch 194/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0012
Epoch 195/200
11314/11314 [==============================] - 29s 3ms/step - loss: 0.0013
Epoch 196/200
11314/11314 [

#### Model Prediction

In [22]:
y_train_predclass = model.predict_classes(x_train_2,batch_size=batch_size)
y_test_predclass = model.predict_classes(x_test_2,batch_size=batch_size)

from sklearn.metrics import accuracy_score, classification_report

print ("Train accuracy:", np.round(accuracy_score(y_train,y_train_predclass), 3))
print ("Test accuracy:", np.round(accuracy_score(y_test,y_test_predclass), 3))

Train accuracy: 1.0
Test accuracy: 0.818
